In [1]:
import numpy as np
import math
import matplotlib as mpl
from qiskit import *
from qiskit.visualization import *
from qiskit.circuit.library import *
from qiskit.providers.aer import QasmSimulator
import qiskit.quantum_info as qi
sim = Aer.get_backend('qasm_simulator')
import  qiskit.circuit.library.grover_operator

In [2]:
aer_sim = Aer.get_backend('aer_simulator')

Implementation of the N=15 a=13 case of Shor's algorithm

In [3]:
# This function is from qiskit textbook
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,7,8,11,13]:
        raise ValueError("'a' must be 2,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a == 11:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

In [4]:
def shors(a,n):
    qc = QuantumCircuit(4+n, n)
    rn = range(n)
    rin = [i+n for i in range(4)]
    for q in range(n):
        qc.h(q)
    qc.x(3+n)
    for q in range(n):
        qc.append(c_amod15(a, 2**q),[q]+rin)
    qc.append(QFT(n,2,inverse=True),rn)
    qc.measure(rn,rn)
    return execute(qc,sim,shots=1024).result().get_counts()

In [5]:
N = 15
a = 13
counts = shors( a, n = 10)
print(counts)
while len(counts)!=0:
    s=counts.popitem()
    r=int(s[0],2)
    if r%2==1:
        continue
    t = int(a**(int(r)/2))
    if t==1:
        continue
    p = math.gcd(t-1,N)
    q = math.gcd(t+1,N)
    if p==1 and q==1:
        continue
    if p==1:
        if q != N:
            print(q,'   ',N//q)
            break
    elif q ==1:
        if p != N:
            print(p,'   ',N//p)
            break
    else:
        print('p = ',p,'   q = ',q)
        break
    print('p = ',p,'   q = ',q)


{'1100000000': 242, '0100000000': 264, '0000000000': 270, '1000000000': 248}
p =  3    q =  5
